In [1]:
from ipywidgets import IntSlider, interact
from Utils import discrete_histogram
import numpy as np
@interact 
def concentration(n=IntSlider(1,1,50,5)):
    import matplotlib.pyplot as plt 
    X= np.random.randint(0,2, size= (n, 10000))*2-1
    means= np.mean(X, axis=0)
    print('P(mean-mu) > 0.5 = %2f <= Chebychev %.5f' % (np.mean(means>0.5), 1/ (0.5**2*n)))
    discrete_histogram(means,normed=True)
    plt.xlim(-1, 1)
    
    

interactive(children=(IntSlider(value=1, description='n', max=50, min=1, step=5), Output()), _dom_classes=('wi…

In [2]:
from ipywidgets import IntSlider, interact
from Utils import discrete_histogram
@interact
def concentration(n=IntSlider(1,1,50,5)):
    import matplotlib.pyplot as plt
    X= np.random.randint(0,2, size=(n, 10000))
    means= np.mean(X, axis=0)
    print('P(mean + u) > 0.5 is %.2f <= Hoeffding %.5f' % (np.mean(means>0.5+0.5), np.exp(-2*n*0.3**2)))
    discrete_histogram(means, normed=True)
    plt.xlim(0,1)

interactive(children=(IntSlider(value=1, description='n', max=50, min=1, step=5), Output()), _dom_classes=('wi…

In [8]:
from ipywidgets import FloatSlider
@interact
def concentration95(n=IntSlider(value=1,min=50,max=500,step=50), p=FloatSlider(value=0.5, min=0,max=1,step=0.1)):
    X= np.random.binomial(1, p, size=(n))
    means= np.mean(X, axis=0)
    epsilon = np.sqrt(-1/(2*n)*np.log((1-0.95)/2))
    print("95%% confidence interval [%.2f, %.2f] for p=%.2f n=%d" % (means-epsilon,means+epsilon,p,n))

interactive(children=(IntSlider(value=50, description='n', max=500, min=50, step=50), FloatSlider(value=0.5, d…

In [13]:
@interact
def concentration300(n=IntSlider(value=1, min=50, max=500, step=50)):
    X=np.minimum(np.abs(70+6*np.random.normal(size=n)), 300)
    means = np.mean(X, axis=0)
    epsilon= np.sqrt(-(300**2)/(2*n)*np.log((1-0.95)/2))
    print("95%% confidence interval [%.2f, %.2f] for n=%.2f" % (means-epsilon,means+epsilon,n))

interactive(children=(IntSlider(value=50, description='n', max=500, min=50, step=50), Output()), _dom_classes=…

In [15]:
def load_sms():
    import csv
    lines=[]
    hamspam = {'ham': 0, 'spam': 1}
    with open('D:/Uppsala Materials/data/spam.csv', mode='r', encoding= 'latin-1') as f:
        reader= csv.reader(f)
        header= next(reader)
        lines= [(line[1], hamspam[line[0]]) for line in reader]
    return lines

In [16]:
sms_data= load_sms()
sms_data[:2]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  0),
 ('Ok lar... Joking wif u oni...', 0)]

In [17]:
interesting_words=set(['free','prize'])
TF10 = {True: 1, False: 0}
Z_obs = [TF10[not interesting_words.isdisjoint([word.lower() for word in line[0].split(' ')])] for line in sms_data]
Y_obs = [y for x,y in sms_data]

In [18]:
def epsilon_bernoulli(n,alpha):
    return np.sqrt(-1/(2*n)*np.log((alpha)/2))

In [19]:
epsilon = epsilon_bernoulli(len(Y_obs),0.05)
mean_Y_obs = np.mean(Y_obs)
print("[%.3f,%.3f]" % (mean_Y_obs-epsilon,mean_Y_obs+epsilon))

[0.116,0.152]


In [24]:
Ymean= np.mean([y for z,y in zip(Z_obs, Y_obs) if z==1])
Ymean

0.812

In [26]:
epi= np.sqrt(-1/(2*len(Y_obs))* np.log((0.05/2)))
print(Ymean-epi, Ymean+epi)

0.7938060632725441 0.830193936727456


In [33]:
def bennetepsilon(n,b,sigma,alpha):
    import scipy.optimize as so
    h= lambda u: (1+u)*np.log(1+u)-u
    f = lambda epsilon: np.exp(-n*sigma**2/b**2*h(b*epsilon/sigma**2))-alpha/2
    ans = so.fsolve(f,0.002)
    epsilon = np.abs(ans[0])
    print("Numerical error", f(epsilon))
    return epsilon

In [34]:
bennetepsilon(50,300,20,0.05)

Numerical error -4.5102810375396984e-17


13.457174445041813

In [36]:
from ipywidgets import FloatSlider
@interact 
def concentration_300(n=IntSlider(value=1,min=50,max=5000,step=50),sigma=FloatSlider(value=10, min=1,max=300,step=1)):
    X = np.minimum(np.abs(70+6*np.random.normal(size=(n))),300)
    means = np.mean(X,axis=0)
    epsilon = np.sqrt(-1/(2*n/(300**2))*np.log((1-0.95)/2))
    epsilon2 = bennetepsilon(n,b=300,sigma=sigma,alpha=0.05)
    print("95%% confidence interval rough [%.2f, %.2f] for n=%d" % (means-epsilon,means+epsilon,n))
    print("95%% confidence interval Bennett [%.2f, %.2f] for n=%d" % (means-epsilon2,means+epsilon2,n))

interactive(children=(IntSlider(value=50, description='n', max=5000, min=50, step=50), FloatSlider(value=10.0,…